In [ ]:
from pathlib import Path
import pandas as pd
# ---- base & result locations ----------------------------------------------
BASE_ROOT    = Path("Self-Ver-In-Conv")     # self-verification result folder
RESULT_ROOT  = Path("result")               # rest of the experiment result folder
DATE_RANGE   = "20220901_20230901"          # constant part of every result path
STRATEGIES = [
    "cot",
    "few_shot",
    "instruc_one_shot",
    "one_shot",
    "zero_shot",
]
# Strategy → column name
COLNAME = {
    "cot": "cot",
    "few_shot": "Instruction-Based",
    "instruc_one_shot": "instruc_one_shot",
    "one_shot": "one_shot",
    "zero_shot": "zero_shot",
}

def load_result_series(model: str, ticker: str, strategy: str) -> pd.Series:
    """
    Fetch the prediction Series for (model, ticker, strategy),
    rename it according to COLNAME, set 'date' as index, and return it.
    """
    run_dir  = (
        RESULT_ROOT
        / f"251_{model}_{model}_on_{DATE_RANGE}_random_{strategy}"
        / ticker.upper()
    )
    csv_path = run_dir / f"{ticker.upper()}_stock_prices_comparison.csv"
    if not csv_path.exists():
        raise FileNotFoundError(csv_path)
    df = pd.read_csv(csv_path)
    
    # Handle different date column names
    date_col = next((col for col in df.columns if col.lower() == 'date'), None)
    if not date_col:
        raise KeyError(f"Date column missing in {csv_path}")
    
    # Try the new column name first, then fall back to the old one
    prediction_col = None
    if 'predicted_price' in df.columns:
        prediction_col = 'predicted_price'
    elif 'a3 stock price' in df.columns:
        prediction_col = 'a3 stock price'
    else:
        raise KeyError(f"Neither 'predicted_price' nor 'a3 stock price' column found in {csv_path}")
    
    return (
        df[[date_col, prediction_col]]
        .rename(columns={date_col: "date", prediction_col: COLNAME[strategy]})
        .set_index("date")[COLNAME[strategy]]
    )

updated = []
# Recursively find every <model>/<model>_<ticker>.csv under Gen_Val_Result_2
for base_path in BASE_ROOT.rglob("*_*.csv"):
    model, ticker_ext = base_path.name.split("_", 1)
    ticker            = ticker_ext.rsplit(".", 1)[0]
    df_base = pd.read_csv(base_path)
    
    # Standardize date column name
    date_col = next((col for col in df_base.columns if col.lower() == 'date'), None)
    if not date_col:
        print(f"[skip] No date column found in {base_path}")
        continue
    
    df_base = df_base.rename(columns={date_col: "date"}).set_index("date")
    
    # Remove legacy zero_shot column if present
    if "zero_shot" in df_base.columns:
        df_base = df_base.drop(columns="zero_shot")
    
    # Append each strategy column
    for strat in STRATEGIES:
        try:
            s = load_result_series(model, ticker, strat)
            df_base = df_base.join(s, how="left")
        except (FileNotFoundError, KeyError) as e:
            print(f"[skip] {e}")
    
    # Re-order columns: actual_price, then strategy columns
    # First find all existing columns we want to order
    ordered_cols = []
    
    # Try to find the actual price column with different possible names
    actual_price_col = None
    for col_name in ['actual_price', 'True Stock Price']:
        if col_name in df_base.columns:
            actual_price_col = col_name
            break
    
    if actual_price_col:
        ordered_cols.append(actual_price_col)
    
    # Add strategy columns that exist
    ordered_cols.extend([COLNAME[s] for s in STRATEGIES if COLNAME[s] in df_base.columns])
    
    # Add any remaining columns not already included
    remaining_cols = [c for c in df_base.columns if c not in ordered_cols]
    ordered_cols.extend(remaining_cols)
    
    # Reorder and save back
    df_base = df_base[ordered_cols]
    
    # Save back
    df_base.reset_index().to_csv(base_path, index=False)
    updated.append(str(base_path.relative_to(BASE_ROOT)))

print("Updated files:")
for f in updated:
    print(" •", f)